### About

In this very simple example, we would like to calculate the Higgs recoil mass on Higgsstrahlung events (e+e- ---> ZH) with Z->mumu.

### Data
Please download these files from [here](https://sas.desy.de/index.php/s/eBYsECq4sN6NGkP):
  * Signal: `E250-TDR_ws.Pe2e2h.Gwhizard-1_95.eL.pR.I106479.001.edm4hep.root`
  * Background: `E250-TDR_ws.P4f_zz_sl.Gwhizard-1_95.eL.pR.I106575.001.edm4hep.root`
  
Create a folder called `data` in the root directory of this repository. Then put into separate foldes `signal`and `bgk`

### Imports

In [2]:
1+1

2

In [5]:
import ROOT
from ROOT import edm4hep
import numpy as np
import os
from os import listdir
from edm4hep_path import get_edm4hep_path
from podio.root_io import Reader

ROOT.gInterpreter.LoadFile(get_edm4hep_path()+"/include/edm4hep/utils/kinematics.h")
USE_ENERGY=edm4hep.utils.detail.UseEnergyTag()

AttributeError: <namespace cppyy.gbl.edm4hep at 0x5f645fdebdd0> has no attribute 'utils'. Full details:
  type object 'edm4hep' has no attribute 'utils'
  'edm4hep::utils' is not a known C++ class
  'utils' is not a known C++ template
  'utils' is not a known C++ enum

### Event Loop

In [ ]:
def doEvtLoop(inputfiles,nbin,xmin,xmax,histtag,title):
    
    reader = Reader(inputfiles)
    hist = ROOT.TH1F(histtag,title,nbin,xmin,xmax)

    pxinitial = 0.
    Einitial = 250. # considering resonance here
    angle = 0.007 # crossing angle parameter, change as needed

    pxinitial = Einitial*angle
    Einitial = 2.*np.sqrt((Einitial/2.)**2 + (pxinitial/2.)**2)
        
    ecms = edm4hep.LorentzVectorE(pxinitial,0.,0.,Einitial)


    for i, event in enumerate(reader.get('events')):            
        
        ## HANDS-ON!!
        # We need to get the muon collection. 
        # Then we should put a cut: Take events with exactly TWO muons
        # Use edmp4hep utils p4: This will put mu1 and mu2 into 4-vector
 
        
        # recoil mass calculations
        recoil = ecms - (mu1 + mu2)
        hist.Fill(recoil.M())
        
    return hist



### Running over samples

In [ ]:
signalDir = '../data/signal/E250-TDR_ws.Pe2e2h.Gwhizard-1_95.eL.pR.I106479.001.edm4hep.root'
bkgDir = '../data/bgk/E250-TDR_ws.P4f_zz_sl.Gwhizard-1_95.eL.pR.I106575.001.edm4hep.root'

sig_hist = doEvtLoop(signalDir,40,50.,250.,"signal","; mass [GeV]; ; ")
bkg_hist = doEvtLoop(bkgDir,40,50.,250.,"bkg", "; mass [GeV]; ; ")

### Draw via ROOT

In [ ]:
c1 = ROOT.TCanvas()
sig_hist.Draw()
hs = ROOT.THStack("hs","; mass [GeV]; ; ")
leg = ROOT.TLegend(0.5,0.6,0.9,0.9)
leg.SetHeader("ILC at 250 GeV")
sig_hist.SetFillColor(2)
leg.AddEntry(sig_hist, "Signal", "F")
bkg_hist.SetFillColor(4)
leg.AddEntry(bkg_hist, "Bkg.","F")
hs.Add(bkg_hist)
hs.Add(sig_hist)
hs.Draw()
leg.Draw()
c1.Draw()
